In [1]:
library(dplyr)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




# Read in data

In [2]:
data=read.csv('/dagher/dagher11/filip/UPF/data/dataset_excluded.csv', quote='"')

# Coalesce food data columns

In [3]:
data_food=data.frame(data[grep('intake',colnames(data))],data[grep('buttermargarine',colnames(data))],
                     data[grep('coffee',colnames(data))], data$eid)

In [4]:
coalesce_set <- function(data, prefix) {
  cols <- grep(paste0("^", prefix), names(data), value = TRUE)
  ordered_cols <- sort(cols, decreasing = TRUE) 
  #print(ordered_cols)
  result <- coalesce(!!!as.list(data[ordered_cols]))
  return(result)
}

In [5]:
unique_prefixes <- unique(sub("_[0-9]+\\..*", "", names(data_food)))

for (prefix in unique_prefixes[-1]) {
  data[[paste0(prefix, "_coalesced")]] <- coalesce_set(data_food, prefix)
}

# Create df with only coalesced data and save it for further analyses

In [6]:
data_food=data[grep('coalesced',colnames(data))]

In [7]:
write.table(data_food,'/dagher/dagher11/filip/UPF/data/coalesced_intake.csv',sep=',',quote=TRUE)